In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

In [28]:
path ="~/data/crude_oil_price/"
# df = pd.read_excel("../../data/crude_oil_price/11_01_2017_11_01_2018.xlsx",sheet_name=None)
# # df = pd.read_csv("../../data/crude_oil_price/11_01_2017_11_01_2018.csv")
# df = df['11_01_2017_11_01_2018']
df = pd.read_csv("../../data/crude_oil_price/20131019_20181119.csv")
df = df.rename({"日期":"date","最新":"latest","開市":"open","高":"high","低":"low","成交量":"quantity","更改%":"percentage"}, axis='columns')
df['percentage'] = [float(x[:-1]) for x in df['percentage']]
opens=df.open
closes=df.latest
highs=df.high
lows=df.low
df.date = pd.to_datetime(df.date, format='%Y年%m月%d日')
df.sort_values(by="date",ascending=True,inplace=True)
std = df['percentage'].std()
avg = df['percentage'].mean()
print("平均:",avg,"標準差:",std)

平均: -0.01946148092744951 標準差: 2.1771093791314886


In [30]:
df.to_csv("../../data/crude_oil_price/history_price_clean_from2013.csv",index = False)

## std_change

In [26]:
days_before = 5
days_after = 2
std_before = df['latest'].rolling(days_before).std().shift(1)
std_after = df['latest'].rolling(days_after).std().shift(-1*(days_after))

latest_pct_change=df['latest'].pct_change(2).shift(-2)
df['two_day_percentage']=latest_pct_change

effective_news_date = df[std_after>std_before]

## pct_change

In [35]:
latest_pct_change=df['latest'].pct_change(2).shift(-2)
latest_pct_change_mean=latest_pct_change.mean()
latest_pct_change_std=latest_pct_change.std()

t=.8
df['two_day_percentage']=latest_pct_change
effective_news_date=df.loc[latest_pct_change>latest_pct_change_mean+t*latest_pct_change_std].set_index('date')
# effective_news_date['two_day_percentage']=latest_pct_change.values
print(len(effective_news_date))
effective_news_date["2018"].head()

211


,latest,open,high,low,quantity,percentage,two_day_percentage
date,,,,,,,
2018-01-02,60.37,60.20,60.74,60.10,510.31K,0.22,0.027166
2018-01-05,61.44,61.90,62.04,61.09,563.04K,-0.92,0.024740
2018-01-08,61.73,61.61,61.97,61.34,616.47K,0.47,0.029807
2018-01-22,63.49,63.61,64.14,63.17,40.06K,0.19,0.033391
2018-02-13,59.19,59.35,59.73,58.39,580.60K,-0.17,0.036324


## Write to csv

In [12]:
effective_news_date.to_csv("../../data/crude_oil_price/effective_news_date_days_before_and_after.csv")

## Plot

In [1]:
# plotly 
import plotly as py
import plotly.graph_objs as go
%load_ext autoreload
%autoreload 2

# offline plot  
# 因為如果寫成func去調用plotly會無法出現，所以只好用offline的方式。
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
# Create traces
trace1 = go.Scatter(
    x = df['date'],
    y = df['percentage'],
    mode = 'lines+markers',
    name = 'lines+markers'
)
trace2 = go.Scatter(
    x = df['date'],
    y = df['open'],
    mode = 'lines+markers',
    name = 'lines+markers'
)

data = [trace1,trace2]
iplot(data, filename='scatter-mode')

NameError: name 'df' is not defined